In [8]:
from dataclasses import dataclass
from typing import Protocol


@dataclass(frozen=True)
class UserInput:
    age: int
    name: str

class ParseError(Exception):
    """Error al convertir el texto de entrada a una estructura intermedia."""


class ValidationError(Exception):
    """Error cuando los datos no cumplen las reglas de negocio."""


class InputParser(Protocol):
    def parse(self, raw: str) -> tuple[int, str]:
        """Convierte la cadena cruda en una tupla."""


class InputValidator(Protocol):
    def validate(self, age: int, name: str) -> UserInput:
        """Valida los campos y retorna un UserInput."""


class DecisionPolicy(Protocol):
    def is_ok(self, user_input: UserInput) -> bool:
        """Regla de decisión principal."""


class CSVParser(InputParser):
    def parse(self, raw: str) -> tuple[int, str]:
        if raw is None:
            raise ParseError("La entrada no puede ser None.")
        parts = [p.strip() for p in raw.split(",")]
        if len(parts) < 2:
            raise ParseError("Formato inválido. Use: edad,nombre")
        age_str, name = parts[0], parts[1]
        if not age_str.isdigit() and not (age_str.startswith("-") and age_str[1:].isdigit()):
            raise ParseError("La edad debe ser un número entero.")
        try:
            age = int(age_str)
        except ValueError as exc:
            raise ParseError("No fue posible convertir la edad a entero.") from exc
        return age, name


class BasicValidator(InputValidator):
    MIN_AGE = 1
    MAX_AGE = 149

    def validate(self, age: int, name: str) -> UserInput:
        if not isinstance(age, int):
            raise ValidationError("La edad debe ser un entero.")
        if not (self.MIN_AGE <= age <= self.MAX_AGE):
            raise ValidationError(f"La edad debe estar entre {self.MIN_AGE} y {self.MAX_AGE}.")
        if not isinstance(name, str) or not name.strip():
            raise ValidationError("El nombre es obligatorio.")
        return UserInput(age=age, name=name.strip())

class DefaultDecisionPolicy(DecisionPolicy):

    def is_ok(self, user_input: UserInput) -> bool:
        return True

class InputProcessor:
    def __init__(
        self,
        parser: InputParser,
        validator: InputValidator,
        policy: DecisionPolicy
    ) -> None:
        self._parser = parser
        self._validator = validator
        self._policy = policy

    def process(self, raw: str) -> str:
        age, name = self._parser.parse(raw)
        user_input = self._validator.validate(age, name)
        return "OK" if self._policy.is_ok(user_input) else "Error"

def main() -> None:
    parser = CSVParser()
    validator = BasicValidator()
    policy = DefaultDecisionPolicy()
    processor = InputProcessor(parser, validator, policy)

    try:
        raw = input("Ingrese datos en formato 'edad,nombre': ").strip()
        result = processor.process(raw)
        print(result)
    except (ParseError, ValidationError) as e:
        print(f"Error: {e}")
    except Exception as e:
        print("Error inesperado. Contacte al administrador.")

if __name__ == "__main__":
    main()


Ingrese datos en formato 'edad,nombre': 12,asda
OK


In [5]:
from __future__ import annotations
from abc import ABC, abstractmethod
from dataclasses import dataclass

class NotificationSender(ABC):
    @abstractmethod
    def send(self, message: str, recipient: str) -> None: ...

class EmailNotification(NotificationSender):
    def send(self, message: str, recipient: str) -> None:
        print(f"[EMAIL] To: {recipient} | Msg: {message}")


class SMSNotification(NotificationSender):
    def send(self, message: str, recipient: str) -> None:
        print(f"[SMS]   To: {recipient} | Msg: {message}")


class PushNotification(NotificationSender):
    def send(self, message: str, recipient: str) -> None:
        print(f"[PUSH]  To: {recipient} | Msg: {message}")

@dataclass(frozen=True)
class Notification:
    recipient: str
    message: str

class NotificationService:
    def __init__(self, sender: NotificationSender) -> None:
        self._sender = sender

    def notify(self, notification: Notification) -> None:
        self._validate(notification)
        self._sender.send(notification.message, notification.recipient)

    @staticmethod
    def _validate(n: Notification) -> None:
        if not n.recipient or not isinstance(n.recipient, str):
            raise ValueError("recipient es requerido.")
        if not n.message or not isinstance(n.message, str):
            raise ValueError("message es requerido.")

class SenderFactory:
    _registry: dict[str, type[NotificationSender]] = {
        "email": EmailNotification,
        "sms": SMSNotification,
        "push": PushNotification,
    }

    @classmethod
    def create(cls, channel: str) -> NotificationSender:
        try:
            return cls._registry[channel.lower()]()
        except KeyError:
            raise ValueError(f"Canal no soportado: {channel!r}. Registrado: {list(cls._registry)}")

    @classmethod
    def register(cls, name: str, sender_cls: type[NotificationSender]) -> None:
        cls._registry[name.lower()] = sender_cls

if __name__ == "__main__":
    email_service = NotificationService(SenderFactory.create("email"))
    sms_service   = NotificationService(SenderFactory.create("sms"))
    push_service  = NotificationService(SenderFactory.create("push"))

    email_service.notify(Notification(recipient="est_s_rueda@fesc.edu.co", message="Informe enviado."))
    sms_service.notify(Notification(recipient="3125108134", message="Código: 50147"))
    push_service.notify(Notification(recipient="user", message="Nueva tarea asignada."))



[EMAIL] To: est_s_rueda@fesc.edu.co | Msg: Informe enviado.
[SMS]   To: 3125108134 | Msg: Código: 50147
[PUSH]  To: user | Msg: Nueva tarea asignada.
